# AI Project Sub3

In [2]:
from konlpy.tag import Kkma
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
from configs import DEFINES

In [3]:
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]

# Req 1-1-1. 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리

In [ ]:
def load_data(file):
    datas = pd.read_csv(file, skiprows=[0,0], header=None)
    q = datas[0]
    a = datas[1]
    train_q, test_q, train_a, test_a = train_test_split(q, a, test_size=0.35, random_state=321)
    
    return train_q, train_a, test_q, test_a

In [ ]:
train_q, train_a, test_q, test_a = load_data("./data_in/ChatBotData.csv")